# 📚 RAG & Document Processing - 문서 기반 지식 응답

이 노트북은 beanllm의 RAG (Retrieval-Augmented Generation) 기능과 문서 처리 기능을 학습합니다.

## 📋 목차

- [1. 문서 로딩](#1-문서-로딩)
- [2. 텍스트 분할 (Chunking)](#2-텍스트-분할-chunking)
- [3. RAGChain 기본 사용](#3-ragchain-기본-사용)
- [4. 고급 RAG 기능](#4-고급-rag-기능)
- [5. 벡터 스토어 선택](#5-벡터-스토어-선택)
- [6. 실전 예제](#6-실전-예제)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ 다양한 형식의 문서를 로드할 수 있습니다
- ✅ 텍스트를 적절히 분할할 수 있습니다
- ✅ RAGChain을 사용하여 문서 기반 질의응답을 할 수 있습니다
- ✅ 배치 쿼리와 스트리밍을 활용할 수 있습니다
- ✅ 다양한 벡터 스토어를 선택할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- [02_core_client.ipynb](02_core_client.ipynb) 완료
- 최소 하나의 LLM Provider API 키 설정


## 1. 문서 로딩

beanllm은 다양한 형식의 문서를 로드할 수 있습니다. 문서를 로드하면 `Document` 객체로 변환되어, 이후 RAG 파이프라인에서 사용할 수 있습니다.

> 💡 **Document 객체**:
> - `content`: 문서의 텍스트 내용
> - `metadata`: 문서의 메타데이터 (소스, 페이지 번호 등)
> - 다양한 로더를 통해 자동으로 생성됩니다

### 1.1 텍스트 파일 로딩

가장 기본적인 텍스트 파일 로딩입니다. `.txt` 파일을 읽어서 `Document` 객체로 변환합니다.

> 💡 **텍스트 파일 로딩의 특징**:
> - 가장 간단하고 빠른 방법
> - 인코딩 자동 감지
> - 메타데이터에 파일 경로 자동 저장


In [ ]:
from beanllm import TextLoader, Document
from pathlib import Path

# ============================================
# 텍스트 파일 로딩
# ============================================

print("=" * 60)
print("텍스트 파일 로딩")
print("=" * 60)

# 방법 1: TextLoader 사용
print("\n[방법 1] TextLoader 사용:")
# loader = TextLoader("sample.txt")
# documents = loader.load()
#
# print(f"  ✅ 로드된 문서 수: {len(documents)}")
# for i, doc in enumerate(documents, 1):
#     print(f"\n  📄 문서 {i}:")
#     print(f"    - 내용 길이: {len(doc.content)}자")
#     print(f"    - 내용 미리보기: {doc.content[:100]}...")
#     print(f"    - 메타데이터: {doc.metadata}")

# 방법 2: 직접 Document 생성
print("\n[방법 2] 직접 Document 생성:")
doc = Document(
    content="Python은 1991년에 발표된 고급 프로그래밍 언어입니다. "
    "간결하고 읽기 쉬운 문법이 특징입니다. "
    "다양한 분야에서 활용되며, 특히 데이터 과학과 웹 개발에 널리 사용됩니다.",
    metadata={
        "source": "manual",
        "type": "introduction",
        "language": "ko",
        "created_at": "2025-01-01",
    },
)

print("  ✅ Document 생성 완료!")
print(f"  📄 내용: {doc.content}")
print(f"  📊 메타데이터: {doc.metadata}")

# 방법 3: 여러 텍스트 파일 일괄 로딩
print("\n[방법 3] 여러 텍스트 파일 일괄 로딩:")
# text_files = ["file1.txt", "file2.txt", "file3.txt"]
# all_documents = []
#
# for file_path in text_files:
#     try:
#         loader = TextLoader(file_path)
#         docs = loader.load()
#         all_documents.extend(docs)
#         print(f"  ✅ {file_path}: {len(docs)}개 문서 로드")
#     except Exception as e:
#         print(f"  ❌ {file_path}: 로드 실패 - {e}")
#
# print(f"\n  📊 총 로드된 문서: {len(all_documents)}개")

print("\n💡 텍스트 파일 로딩 팁:")
print("  - 인코딩 문제 시 encoding 파라미터 지정")
print("  - 큰 파일은 청크 단위로 처리 고려")
print("  - 메타데이터에 파일 정보 자동 저장")
print("  - 여러 파일을 로드하여 하나의 Document 리스트로 통합 가능")

# ============================================
# Document 객체 활용
# ============================================

print("\n" + "=" * 60)
print("Document 객체 활용")
print("=" * 60)

# Document 속성 확인
print("\n📊 Document 객체 구조:")
print(f"  - content: {type(doc.content).__name__} (길이: {len(doc.content)}자)")
print(f"  - metadata: {type(doc.metadata).__name__} (키: {list(doc.metadata.keys())})")

# 메타데이터 활용
print("\n💡 메타데이터 활용 예시:")
print("  - 문서 출처 추적")
print("  - 문서 필터링 (예: 특정 타입만 선택)")
print("  - 문서 그룹화 (예: 소스별 그룹)")
print("  - 검색 결과에 출처 표시")

print("\n" + "=" * 60)
print("✅ 텍스트 파일 로딩 학습 완료!")
print("=" * 60)

### 1.2 PDF 파일 로딩

PDF 파일을 로드할 수 있습니다. beanllm은 여러 PDF 처리 엔진을 지원하여 다양한 PDF 형식을 처리할 수 있습니다.

> 💡 **PDF 로딩의 특징**:
> - 텍스트 추출: PDF에서 텍스트를 추출하여 Document로 변환
> - 페이지 단위 처리: 각 페이지를 별도의 Document로 처리 가능
> - 메타데이터 보존: 페이지 번호, 파일 경로 등 메타데이터 자동 저장
> - 이미지 및 표 처리: 고급 PDF 처리 엔진 사용 시 이미지와 표도 추출 가능

> ⚠️ **주의사항**:
> - 스캔된 PDF는 OCR이 필요할 수 있습니다
> - 복잡한 레이아웃의 PDF는 추가 처리가 필요할 수 있습니다


In [ ]:
from beanllm import PDFLoader
from pathlib import Path

# ============================================
# PDF 파일 로딩
# ============================================

print("=" * 60)
print("PDF 파일 로딩")
print("=" * 60)

# 방법 1: 기본 PDFLoader 사용
print("\n[방법 1] 기본 PDFLoader 사용:")
# loader = PDFLoader("document.pdf")
# documents = loader.load()
#
# print(f"  ✅ PDF 문서 로드: {len(documents)}개 페이지")
# for i, doc in enumerate(documents[:3], 1):  # 처음 3페이지만
#     print(f"\n  📄 페이지 {i}:")
#     print(f"    - 내용 길이: {len(doc.content)}자")
#     print(f"    - 내용 미리보기: {doc.content[:150]}...")
#     print(f"    - 메타데이터: {doc.metadata}")

# 방법 2: 페이지 범위 지정
print("\n[방법 2] 페이지 범위 지정:")
# loader = PDFLoader("document.pdf", page_range=(1, 5))  # 1-5페이지만
# documents = loader.load()
# print(f"  ✅ 로드된 페이지: {len(documents)}개")

# 방법 3: 고급 PDF 처리 (beanPDFLoader)
print("\n[방법 3] 고급 PDF 처리 (beanPDFLoader):")
print("  from beanllm.domain.pdf import beanPDFLoader")
print("  ")
print("  loader = beanPDFLoader(")
print("      source='document.pdf',")
print("      extract_images=True,  # 이미지 추출")
print("      extract_tables=True,  # 표 추출")
print("      ocr_enabled=True      # OCR 활성화")
print("  )")
print("  documents = loader.load()")

print("\n💡 PDF 로딩 옵션:")
print("  - page_range: 특정 페이지 범위만 로드")
print("  - extract_images: 이미지 추출 여부")
print("  - extract_tables: 표 추출 여부")
print("  - ocr_enabled: OCR 활성화 (스캔된 PDF용)")

# ============================================
# PDF 메타데이터 활용
# ============================================

print("\n" + "=" * 60)
print("PDF 메타데이터 활용")
print("=" * 60)

# PDF 문서의 메타데이터 예시
print("\n📊 PDF Document 메타데이터 구조:")
pdf_metadata_example = {
    "source": "document.pdf",
    "page": 1,
    "total_pages": 10,
    "file_size": 1024000,
    "created_at": "2025-01-01",
    "author": "작성자 이름",
}

print("  예시 메타데이터:")
for key, value in pdf_metadata_example.items():
    print(f"    - {key}: {value}")

print("\n💡 메타데이터 활용:")
print("  - 페이지 번호로 특정 페이지 참조")
print("  - 출처 추적 (어떤 PDF의 몇 페이지인지)")
print("  - 문서 필터링 (예: 특정 페이지 범위만 사용)")
print("  - 검색 결과에 페이지 번호 표시")

print("\n" + "=" * 60)
print("✅ PDF 파일 로딩 학습 완료!")
print("=" * 60)

### 1.3 디렉토리 로딩

디렉토리 내의 모든 파일을 한 번에 로드할 수 있습니다. 병렬 처리도 지원하여 대량의 문서를 빠르게 처리할 수 있습니다.

> 💡 **디렉토리 로딩의 특징**:
> - 재귀적 검색: 하위 디렉토리까지 자동으로 검색
> - 파일 타입 필터링: glob 패턴으로 특정 파일만 로드
> - 병렬 처리: 여러 파일을 동시에 처리하여 속도 향상
> - 메타데이터 자동 저장: 파일 경로, 타입 등 자동으로 메타데이터에 저장

> ⚠️ **주의사항**:
> - 큰 디렉토리는 메모리 사용량이 많을 수 있습니다
> - 병렬 처리는 CPU 코어 수에 따라 조정하세요


In [ ]:
from beanllm import DirectoryLoader
from pathlib import Path

# ============================================
# 디렉토리 로딩
# ============================================

print("=" * 60)
print("디렉토리 로딩")
print("=" * 60)

# 방법 1: 기본 디렉토리 로딩
print("\n[방법 1] 기본 디렉토리 로딩:")
# loader = DirectoryLoader(
#     path="./docs",
#     glob="**/*.txt",  # 모든 .txt 파일
#     recursive=True,   # 재귀적 검색
# )
# documents = loader.load()
# print(f"  ✅ 로드된 문서: {len(documents)}개")

# 방법 2: 병렬 처리 활성화
print("\n[방법 2] 병렬 처리 활성화:")
# loader = DirectoryLoader(
#     path="./docs",
#     glob="**/*.{txt,md,pdf}",  # 여러 파일 타입
#     recursive=True,
#     use_parallel=True,  # 병렬 처리 활성화
#     max_workers=4,      # 워커 수 (CPU 코어 수에 맞게 조정)
# )
# documents = loader.load()
# print(f"  ✅ 로드된 문서: {len(documents)}개 (병렬 처리)")

# 방법 3: 파일 타입별 필터링
print("\n[방법 3] 파일 타입별 필터링:")
# # 텍스트 파일만
# text_loader = DirectoryLoader(
#     path="./docs",
#     glob="**/*.txt",
#     recursive=True
# )
# text_docs = text_loader.load()
#
# # PDF 파일만
# pdf_loader = DirectoryLoader(
#     path="./docs",
#     glob="**/*.pdf",
#     recursive=True
# )
# pdf_docs = pdf_loader.load()
#
# print(f"  ✅ 텍스트 파일: {len(text_docs)}개")
# print(f"  ✅ PDF 파일: {len(pdf_docs)}개")

# 방법 4: 제외 패턴 사용
print("\n[방법 4] 제외 패턴 사용:")
# loader = DirectoryLoader(
#     path="./docs",
#     glob="**/*",
#     recursive=True,
#     exclude=["**/node_modules/**", "**/.git/**", "**/__pycache__/**"]  # 제외할 패턴
# )
# documents = loader.load()
# print(f"  ✅ 로드된 문서: {len(documents)}개 (제외 패턴 적용)")

print("\n💡 DirectoryLoader 옵션:")
print("  - path: 로드할 디렉토리 경로")
print("  - glob: 파일 패턴 (예: '**/*.txt', '**/*.{txt,md}')")
print("  - recursive: 하위 디렉토리 검색 여부")
print("  - use_parallel: 병렬 처리 활성화")
print("  - max_workers: 병렬 처리 워커 수")
print("  - exclude: 제외할 파일 패턴 리스트")

# ============================================
# 디렉토리 로딩 결과 분석
# ============================================

print("\n" + "=" * 60)
print("디렉토리 로딩 결과 분석")
print("=" * 60)

# 실제 사용 예제 (주석 해제)
# documents = DirectoryLoader("./docs", glob="**/*", recursive=True).load()
#
# print(f"\n📊 로딩 통계:")
# print(f"  - 총 문서 수: {len(documents)}개")
#
# # 파일 타입별 분류
# file_types = {}
# for doc in documents:
#     source = doc.metadata.get("source", "unknown")
#     file_ext = Path(source).suffix.lower() if source != "unknown" else "unknown"
#     file_types[file_ext] = file_types.get(file_ext, 0) + 1
#
# print(f"\n📁 파일 타입별 분류:")
# for file_type, count in sorted(file_types.items(), key=lambda x: x[1], reverse=True):
#     print(f"  - {file_type}: {count}개")
#
# # 디렉토리별 분류
# directories = {}
# for doc in documents:
#     source = doc.metadata.get("source", "unknown")
#     if source != "unknown":
#         dir_path = str(Path(source).parent)
#         directories[dir_path] = directories.get(dir_path, 0) + 1
#
# print(f"\n📂 디렉토리별 분류 (상위 5개):")
# for dir_path, count in sorted(directories.items(), key=lambda x: x[1], reverse=True)[:5]:
#     print(f"  - {dir_path}: {count}개")

print("\n💡 디렉토리 로딩 활용:")
print("  - 대량 문서 일괄 처리")
print("  - 문서 컬렉션 구축")
print("  - RAG 시스템 데이터 준비")
print("  - 문서 분석 및 통계")

print("\n" + "=" * 60)
print("✅ 디렉토리 로딩 학습 완료!")
print("=" * 60)

### 1.4 다양한 문서 형식

beanllm은 다양한 문서 형식을 지원합니다. 각 형식에 맞는 로더를 사용하면 자동으로 텍스트를 추출하고 Document 객체로 변환합니다.

> 💡 **지원하는 문서 형식**:
> - **텍스트 파일**: .txt, .md, .rst 등
> - **PDF**: .pdf (텍스트, 이미지, 표 추출 가능)
> - **CSV**: .csv (테이블 데이터)
> - **HTML**: .html, .htm (웹 페이지)
> - **Markdown**: .md, .markdown
> - **Jupyter Notebook**: .ipynb
> - **Office 문서**: .docx, .xlsx, .pptx (고급 로더 사용 시)
> - **이미지**: .jpg, .png 등 (OCR 또는 Vision 모델 사용 시)

> ⚠️ **주의사항**:
> - 일부 형식은 추가 패키지 설치가 필요할 수 있습니다
> - 복잡한 레이아웃의 문서는 추가 처리가 필요할 수 있습니다


In [ ]:
from beanllm import TextLoader, PDFLoader, CSVLoader, HTMLLoader, MarkdownLoader, Document

# ============================================
# 다양한 문서 형식 로딩
# ============================================

print("=" * 60)
print("다양한 문서 형식 로딩")
print("=" * 60)

# 지원하는 문서 형식 및 로더
supported_formats = {
    "텍스트": {
        "extensions": [".txt", ".text"],
        "loader": "TextLoader",
        "description": "기본 텍스트 파일",
        "example": 'TextLoader("file.txt").load()',
    },
    "PDF": {
        "extensions": [".pdf"],
        "loader": "PDFLoader, beanPDFLoader",
        "description": "PDF 문서 (텍스트, 이미지, 표 추출 가능)",
        "example": 'PDFLoader("file.pdf").load()',
    },
    "CSV": {
        "extensions": [".csv"],
        "loader": "CSVLoader",
        "description": "CSV 테이블 데이터",
        "example": 'CSVLoader("data.csv").load()',
    },
    "HTML": {
        "extensions": [".html", ".htm"],
        "loader": "HTMLLoader",
        "description": "HTML 웹 페이지",
        "example": 'HTMLLoader("page.html").load()',
    },
    "Markdown": {
        "extensions": [".md", ".markdown"],
        "loader": "MarkdownLoader",
        "description": "Markdown 문서",
        "example": 'MarkdownLoader("readme.md").load()',
    },
    "Jupyter Notebook": {
        "extensions": [".ipynb"],
        "loader": "JupyterLoader",
        "description": "Jupyter 노트북 (코드 및 마크다운)",
        "example": 'JupyterLoader("notebook.ipynb").load()',
    },
    "Office 문서": {
        "extensions": [".docx", ".xlsx", ".pptx"],
        "loader": "DoclingLoader (고급)",
        "description": "Word, Excel, PowerPoint (고급 로더 필요)",
        "example": 'DoclingLoader("document.docx").load()',
    },
}

print("\n📚 지원하는 문서 형식:")
for format_name, info in supported_formats.items():
    print(f"\n[{format_name}]")
    print(f"  확장자: {', '.join(info['extensions'])}")
    print(f"  로더: {info['loader']}")
    print(f"  설명: {info['description']}")
    print(f"  예제: {info['example']}")

# ============================================
# 실제 로딩 예제
# ============================================

print("\n" + "=" * 60)
print("실제 로딩 예제")
print("=" * 60)

# CSV 파일 로딩 예제
print("\n[예제 1] CSV 파일 로딩:")
# loader = CSVLoader("data.csv")
# documents = loader.load()
# print(f"  ✅ 로드된 행 수: {len(documents)}개")
# for i, doc in enumerate(documents[:3], 1):
#     print(f"  행 {i}: {doc.content[:100]}...")

# HTML 파일 로딩 예제
print("\n[예제 2] HTML 파일 로딩:")
# loader = HTMLLoader("page.html")
# documents = loader.load()
# print(f"  ✅ 로드된 문서: {len(documents)}개")
# for doc in documents:
#     print(f"  내용 미리보기: {doc.content[:150]}...")

# Markdown 파일 로딩 예제
print("\n[예제 3] Markdown 파일 로딩:")
# loader = MarkdownLoader("readme.md")
# documents = loader.load()
# print(f"  ✅ 로드된 문서: {len(documents)}개")
# print(f"  내용: {documents[0].content[:200]}...")

print("\n💡 문서 형식별 특징:")
print("  - 텍스트: 가장 간단하고 빠름")
print("  - PDF: 레이아웃 보존, 이미지/표 추출 가능")
print("  - CSV: 테이블 데이터 구조화")
print("  - HTML: 웹 콘텐츠 추출")
print("  - Markdown: 구조화된 텍스트")
print("  - Jupyter: 코드와 설명 분리")

print("\n" + "=" * 60)
print("✅ 다양한 문서 형식 로딩 학습 완료!")
print("=" * 60)

## 2. 텍스트 분할 (Chunking)

긴 문서를 작은 청크로 나누는 것은 RAG의 핵심입니다. 적절한 청크 크기와 오버랩을 설정하면 검색 정확도와 응답 품질을 크게 향상시킬 수 있습니다.

> 💡 **청킹의 중요성**:
> - **너무 작으면**: 컨텍스트 손실, 의미 파편화
> - **너무 크면**: 검색 정확도 저하, 불필요한 정보 포함
> - **적절한 크기**: 일반적으로 200-1000자 또는 100-500 토큰
> - **오버랩**: 문맥 유지, 경계에서 정보 손실 방지 (일반적으로 10-20%)

> 💡 **청킹 전략**:
> - **문장 단위**: 문장을 기준으로 분할 (의미 보존)
> - **단락 단위**: 단락을 기준으로 분할 (논리적 단위)
> - **토큰 단위**: 토큰 수를 기준으로 분할 (모델 제한 고려)
> - **구조 기반**: 마크다운 헤더, HTML 태그 등 구조를 활용


In [ ]:
from beanllm import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
    MarkdownHeaderTextSplitter,
    Document
)

# ============================================
# 기본 텍스트 분할
# ============================================

print("=" * 60)
print("텍스트 분할 (Chunking)")
print("=" * 60)

# 긴 텍스트 예제
long_text = """
Python은 1991년에 발표된 고급 프로그래밍 언어입니다.
간결하고 읽기 쉬운 문법이 특징입니다.

Python의 주요 특징:
1. 간결한 문법: 코드가 읽기 쉽고 작성하기 쉽습니다.
2. 다양한 라이브러리: 수많은 표준 라이브러리와 서드파티 패키지가 있습니다.
3. 크로스 플랫폼: Windows, macOS, Linux 등 다양한 운영체제에서 실행됩니다.
4. 동적 타입: 변수 타입을 명시적으로 선언할 필요가 없습니다.
5. 객체지향: 클래스와 객체를 사용한 프로그래밍을 지원합니다.

Python은 데이터 과학, 웹 개발, 자동화, 인공지능 등 다양한 분야에서 사용됩니다.
특히 데이터 분석과 머신러닝 분야에서 매우 인기 있는 언어입니다.
"""

print(f"\n📄 원본 텍스트 길이: {len(long_text)}자")

# 방법 1: RecursiveCharacterTextSplitter (권장)
print("\n[방법 1] RecursiveCharacterTextSplitter (권장):")
splitter1 = RecursiveCharacterTextSplitter(
    chunk_size=200,      # 청크 크기 (문자 수)
    chunk_overlap=50,    # 오버랩 (문자 수)
    length_function=len, # 길이 측정 함수
    separators=["\n\n", "\n", ". ", " ", ""]  # 분할 우선순위
)

# chunks = splitter1.split_text(long_text)
# print(f"  ✅ 생성된 청크 수: {len(chunks)}개")
# for i, chunk in enumerate(chunks, 1):
#     print(f"\n  청크 {i} ({len(chunk)}자):")
#     print(f"    {chunk[:100]}...")

print("\n💡 RecursiveCharacterTextSplitter 특징:")
print("  - 문단, 문장, 단어 순서로 분할 시도")
print("  - 의미 단위 보존")
print("  - 가장 일반적으로 사용되는 방법")

# 방법 2: CharacterTextSplitter (간단한 분할)
print("\n[방법 2] CharacterTextSplitter (간단한 분할):")
splitter2 = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separator="\n"  # 줄바꿈으로 분할
)

# chunks = splitter2.split_text(long_text)
# print(f"  ✅ 생성된 청크 수: {len(chunks)}개")

print("\n💡 CharacterTextSplitter 특징:")
print("  - 지정된 구분자로 단순 분할")
print("  - 빠른 처리 속도")
print("  - 의미 보존은 낮음")

# 방법 3: TokenTextSplitter (토큰 단위 분할)
print("\n[방법 3] TokenTextSplitter (토큰 단위 분할):")
splitter3 = TokenTextSplitter(
    chunk_size=100,      # 청크 크기 (토큰 수)
    chunk_overlap=20,    # 오버랩 (토큰 수)
    model="gpt-4o"      # 토큰 계산에 사용할 모델
)

# chunks = splitter3.split_text(long_text)
# print(f"  ✅ 생성된 청크 수: {len(chunks)}개")

print("\n💡 TokenTextSplitter 특징:")
print("  - 모델의 토큰 제한을 고려")
print("  - 정확한 토큰 수 계산")
print("  - LLM 컨텍스트 윈도우에 최적화")

# 방법 4: MarkdownHeaderTextSplitter (구조 기반 분할)
print("\n[방법 4] MarkdownHeaderTextSplitter (구조 기반 분할):")
markdown_text = """
# Python 소개

Python은 1991년에 발표된 고급 프로그래밍 언어입니다.

## 주요 특징

### 간결한 문법
코드가 읽기 쉽고 작성하기 쉽습니다.

### 다양한 라이브러리
수많은 표준 라이브러리와 서드파티 패키지가 있습니다.
"""

splitter4 = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
)

# chunks = splitter4.split_text(markdown_text)
# print(f"  ✅ 생성된 청크 수: {len(chunks)}개")
# for i, chunk in enumerate(chunks, 1):
#     print(f"\n  청크 {i}:")
#     print(f"    헤더: {chunk.metadata}")
#     print(f"    내용: {chunk.content[:100]}...")

print("\n💡 MarkdownHeaderTextSplitter 특징:")
print("  - 마크다운 헤더를 기준으로 분할")
print("  - 논리적 구조 보존")
print("  - 메타데이터에 헤더 정보 저장")

# ============================================
# 청킹 전략 비교
# ============================================

print("\n" + "=" * 60)
print("청킹 전략 비교")
print("=" * 60)

strategies = {
    "작은 청크 (100자)": {
        "chunk_size": 100,
        "chunk_overlap": 20,
        "장점": "정확한 검색, 빠른 처리",
        "단점": "컨텍스트 손실 가능"
    },
    "중간 청크 (500자)": {
        "chunk_size": 500,
        "chunk_overlap": 50,
        "장점": "균형잡힌 성능",
        "단점": "없음 (권장)"
    },
    "큰 청크 (1000자)": {
        "chunk_size": 1000,
        "chunk_overlap": 100,
        "장점": "맥락 보존",
        "단점": "검색 정확도 저하 가능"
    }
}

print("\n📊 청크 크기별 비교:")
for strategy_name, info in strategies.items():
    print(f"\n[{strategy_name}]")
    print(f"  크기: {info['chunk_size']}자, 오버랩: {info['chunk_overlap']}자")
    print(f"  장점: {info['장점']}")
    print(f"  단점: {info['단점']}")

print("\n💡 청킹 전략 선택 가이드:")
print("  - 일반 문서: 500자, 50자 오버랩 (권장)")
print("  - 기술 문서: 300-500자, 30-50자 오버랩")
print("  - 코드: 200-400자, 20-40자 오버랩")
print("  - 긴 문서: 800-1000자, 80-100자 오버랩")

print("\n" + "=" * 60)
print("✅ 텍스트 분할 학습 완료!")
print("=" * 60)
동적 타입 시스템과 해석형 언어입니다.
다양한 프로그래밍 패러다임을 지원합니다.
객체지향, 함수형, 절차형 프로그래밍이 모두 가능합니다.
풍부한 표준 라이브러리를 제공합니다.
커뮤니티가 크고 활발합니다.
데이터 과학, 웹 개발, 자동화 등 다양한 분야에서 사용됩니다.
"""
    * 5
)  # 5번 반복

# Document 생성
doc = Document(content=long_text.strip())

# 분할
chunks = splitter.split_documents([doc])

print(f"✅ 원본 문서 길이: {len(doc.content)}자")
print(f"✅ 분할된 청크 수: {len(chunks)}개")
print(f"\n📊 청크 크기 통계:")
chunk_sizes = [len(chunk.content) for chunk in chunks]
print(f"  - 평균: {sum(chunk_sizes) / len(chunk_sizes):.1f}자")
print(f"  - 최소: {min(chunk_sizes)}자")
print(f"  - 최대: {max(chunk_sizes)}자")

print(f"\n📄 첫 번째 청크:")
print(f"  {chunks[0].content[:100]}...")


### 2.1 다양한 Splitter

문서 타입에 따라 적절한 Splitter를 선택할 수 있습니다.


In [ ]:
from beanllm import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
    MarkdownHeaderTextSplitter,
)

# 1. RecursiveCharacterTextSplitter (권장)
# 문단, 문장, 단어 순서로 분할
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)
print("✅ RecursiveCharacterTextSplitter: 문단/문장/단어 순서로 분할")

# 2. CharacterTextSplitter
# 단순히 문자 수로 분할
char_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
)
print("✅ CharacterTextSplitter: 단순 문자 수 기준")

# 3. TokenTextSplitter
# 토큰 수 기준으로 분할 (더 정확)
token_splitter = TokenTextSplitter(
    chunk_size=100,  # 토큰 수
    chunk_overlap=20,
)
print("✅ TokenTextSplitter: 토큰 수 기준 (LLM에 최적화)")

# 4. MarkdownHeaderTextSplitter
# Markdown 헤더 기준으로 분할
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "Header 1"), ("##", "Header 2")]
)
print("✅ MarkdownHeaderTextSplitter: Markdown 헤더 기준")

## 3. RAGChain 기본 사용

RAGChain은 문서를 로드하고, 임베딩을 생성하고, 벡터 스토어에 저장한 후 질의응답을 수행하는 모든 과정을 자동화합니다.

### 3.1 원라이너 RAG

가장 간단한 방법은 `from_documents()`를 사용하는 것입니다.


In [ ]:
import asyncio
from beanllm import RAGChain, Document


async def simple_rag():
    """간단한 RAG 예제"""

    # 문서 준비
    documents = [
        Document(content="Python은 1991년에 발표된 프로그래밍 언어입니다."),
        Document(content="Java는 1995년에 발표된 객체지향 프로그래밍 언어입니다."),
        Document(content="JavaScript는 웹 개발을 위한 스크립트 언어입니다."),
    ]

    # RAGChain 생성 (원라이너)
    rag = await RAGChain.from_documents(
        documents=documents,
        embedding_model="text-embedding-3-small",  # 임베딩 모델
        vector_store="chroma",  # 벡터 스토어
    )

    print("✅ RAGChain 생성 완료!")

    # 질문하기
    response = await rag.query("Python이란 무엇인가요?")

    print(f"\n💬 질문: Python이란 무엇인가요?")
    print(f"🤖 응답: {response.answer}")
    print(f"\n📚 참조 문서: {len(response.sources)}개")
    for i, source in enumerate(response.sources[:3], 1):
        print(f"  {i}. {source.content[:50]}... (유사도: {source.score:.3f})")

    return response


# 실행 (주석 해제)
# response = await simple_rag()
print("💡 위의 주석을 해제하여 실행하세요.")

### 3.2 디렉토리에서 RAG 생성

디렉토리의 모든 문서를 자동으로 로드하여 RAG를 생성할 수 있습니다.


In [ ]:
async def rag_from_directory():
    """디렉토리에서 RAG 생성"""

    # 디렉토리 경로에서 직접 RAG 생성
    # rag = await RAGChain.from_documents(
    #     documents_path="./docs",  # 디렉토리 경로
    #     embedding_model="text-embedding-3-small",
    #     vector_store="chroma",
    # )

    # 또는 수동으로 로드 후 생성
    from beanllm import DirectoryLoader

    # loader = DirectoryLoader("./docs", glob="**/*.txt")
    # documents = loader.load()

    # rag = await RAGChain.from_documents(
    #     documents=documents,
    #     embedding_model="text-embedding-3-small",
    #     vector_store="chroma",
    # )

    print("💡 디렉토리에서 RAG 생성:")
    print("1. DirectoryLoader로 문서 로드")
    print("2. RAGChain.from_documents()로 RAG 생성")
    print("3. 자동으로 임베딩 생성 및 벡터 스토어 저장")


# await rag_from_directory()

### 3.3 스트리밍 쿼리

긴 응답을 실시간으로 받을 수 있습니다.


In [ ]:
async def streaming_rag():
    """스트리밍 RAG 쿼리"""

    documents = [
        Document(content="Python은 간결하고 읽기 쉬운 문법이 특징입니다."),
        Document(content="Python은 데이터 과학, 웹 개발 등 다양한 분야에서 사용됩니다."),
    ]

    rag = await RAGChain.from_documents(
        documents=documents,
        embedding_model="text-embedding-3-small",
    )

    print("📡 스트리밍 응답:\n")
    print("🤖 AI: ", end="", flush=True)

    async for chunk in rag.stream_query("Python의 특징을 설명해주세요."):
        print(chunk, end="", flush=True)

    print("\n\n✅ 스트리밍 완료")


# 실행 (주석 해제)
# await streaming_rag()
print("💡 위의 주석을 해제하여 실행하세요.")

## 4. 고급 RAG 기능

### 4.1 배치 쿼리

여러 질문을 한 번에 처리할 수 있습니다.


In [ ]:
async def batch_rag_query():
    """배치 RAG 쿼리"""

    documents = [
        Document(content="Python은 프로그래밍 언어입니다."),
        Document(content="Java는 객체지향 언어입니다."),
        Document(content="JavaScript는 웹 개발 언어입니다."),
    ]

    rag = await RAGChain.from_documents(
        documents=documents,
        embedding_model="text-embedding-3-small",
    )

    # 여러 질문
    questions = [
        "Python이란?",
        "Java란?",
        "JavaScript란?",
    ]

    print("📋 배치 쿼리 시작...")
    responses = await rag.batch_query(questions)

    print(f"\n✅ {len(responses)}개 질문 처리 완료\n")
    for i, (question, response) in enumerate(zip(questions, responses), 1):
        print(f"{i}. 질문: {question}")
        print(f"   응답: {response.answer[:100]}...")
        print()


# 실행 (주석 해제)
# await batch_rag_query()
print("💡 위의 주석을 해제하여 실행하세요.")

### 4.2 커스텀 파라미터

RAGChain의 동작을 세밀하게 제어할 수 있습니다.


In [ ]:
async def custom_rag():
    """커스텀 파라미터 RAG"""

    documents = [
        Document(content="Python은 간결한 문법이 특징입니다."),
        Document(content="Python은 다양한 라이브러리를 제공합니다."),
    ]

    rag = await RAGChain.from_documents(
        documents=documents,
        embedding_model="text-embedding-3-small",
        vector_store="chroma",
        # RAG 파라미터
        top_k=3,  # 검색할 문서 수
        similarity_threshold=0.7,  # 유사도 임계값
        # LLM 파라미터
        llm_model="gpt-4o",
        temperature=0.7,
        max_tokens=500,
    )

    response = await rag.query("Python의 특징은?")
    print(f"✅ 응답: {response.answer}")


# 실행 (주석 해제)
# await custom_rag()
print("💡 위의 주석을 해제하여 실행하세요.")

## 5. 벡터 스토어 선택

다양한 벡터 스토어를 선택할 수 있습니다.

### 5.1 로컬 벡터 스토어

로컬에서 실행되는 벡터 스토어입니다.


In [ ]:
# 로컬 벡터 스토어 옵션
local_stores = {
    "chroma": "ChromaDB - 사용하기 쉬움, 빠른 개발",
    "faiss": "FAISS - Facebook, 매우 빠름",
    "qdrant": "Qdrant - 고성능, 로컬/클라우드 모두 지원",
    "lancedb": "LanceDB - 빠른 벡터 검색",
    "pgvector": "PostgreSQL 확장, SQL과 통합",
}

print("💾 로컬 벡터 스토어:")
for store, description in local_stores.items():
    print(f"  - {store}: {description}")

# 사용 예제
print("\n💡 사용 방법:")
print("rag = await RAGChain.from_documents(")
print("    documents=docs,")
print("    vector_store='chroma',  # 또는 'faiss', 'qdrant' 등")
print(")")

### 5.2 클라우드 벡터 스토어

클라우드 기반 벡터 스토어입니다.


In [ ]:
# 클라우드 벡터 스토어 옵션
cloud_stores = {
    "pinecone": "Pinecone - 관리형 벡터 DB",
    "weaviate": "Weaviate - GraphQL 지원",
    "milvus": "Milvus - 대규모 벡터 검색",
}

print("☁️ 클라우드 벡터 스토어:")
for store, description in cloud_stores.items():
    print(f"  - {store}: {description}")

print("\n💡 클라우드 스토어 사용 시:")
print("  - API 키 설정 필요")
print("  - 네트워크 연결 필요")
print("  - 확장성과 관리 편의성 제공")

## 6. 실전 예제

### 6.1 문서 기반 Q&A 시스템

실제 사용 사례를 구현해봅시다.


In [ ]:
async def qa_system():
    """문서 기반 Q&A 시스템"""

    # 지식 베이스 문서
    knowledge_base = [
        Document(
            content="beanllm은 다양한 LLM Provider를 통일된 인터페이스로 사용할 수 있는 툴킷입니다.",
            metadata={"category": "introduction"},
        ),
        Document(
            content="RAG는 Retrieval-Augmented Generation의 약자로, 문서 검색과 생성 모델을 결합한 기술입니다.",
            metadata={"category": "rag"},
        ),
        Document(
            content="벡터 스토어는 임베딩 벡터를 저장하고 유사도 검색을 수행하는 데이터베이스입니다.",
            metadata={"category": "vector_store"},
        ),
    ]

    # RAG 시스템 생성
    rag = await RAGChain.from_documents(
        documents=knowledge_base,
        embedding_model="text-embedding-3-small",
        vector_store="chroma",
    )

    # 질문 리스트
    questions = [
        "beanllm이란 무엇인가요?",
        "RAG란 무엇인가요?",
        "벡터 스토어는 무엇인가요?",
    ]

    print("📚 문서 기반 Q&A 시스템\n")
    for question in questions:
        response = await rag.query(question)
        print(f"❓ 질문: {question}")
        print(f"💡 답변: {response.answer}")
        print(f"📄 참조: {len(response.sources)}개 문서\n")


# 실행 (주석 해제)
# await qa_system()
print("💡 위의 주석을 해제하여 실행하세요.")

## 7. 다음 단계

RAG와 문서 처리를 마스터했습니다! 다음 튜토리얼로 진행하세요:

### 📚 추천 학습 순서

1. **[04_embeddings_vector_stores.ipynb](04_embeddings_vector_stores.ipynb)** - 임베딩 및 벡터 스토어
   - 임베딩 생성 방법
   - 벡터 스토어 직접 사용
   - 유사도 검색 최적화

2. **[05_agent_and_tools.ipynb](05_agent_and_tools.ipynb)** - Agent 및 Tools
   - Tool을 사용하는 Agent
   - 복잡한 작업 자동화

3. **[08_vision_rag.ipynb](08_vision_rag.ipynb)** - Vision RAG
   - 이미지 기반 RAG
   - 멀티모달 검색

### 🔗 관련 문서

- [API Reference](../API_REFERENCE.md#ragchain) - RAGChain API 상세 문서
- [README.md](../../README.md) - 프로젝트 개요

---

**✅ RAG & Document Processing 완료! 다음 노트북으로 진행하세요.**
